In [1]:
import pandas as pd
from pathlib import Path
import sys
import numpy 

MAIN_PATH = Path().resolve().parent
sys.path.append(str(MAIN_PATH))

from src.utilities import load_config, load_validations, load_fields, load_data, init_log
from src.utilities.logger import log_event

# Carrega as configurações e inicia o log 
df_config = pd.DataFrame([{}])
load_config(df_config)

# inicia o log com o path parametrizado
init_log(df_config.loc[0, "log_path"])

# Carrega a lista de validações
df_validations = pd.DataFrame([{}])
load_validations(df_validations,df_config.loc[0, "eda_config_path"] )

# Carrega a lista de campos a validar
df_fields = pd.DataFrame([{}])
load_fields(df_fields, df_config.loc[0, "eda_config_path"])


,file,table,field,type,subtype,null,pk,fk,table_fk,format,Range,values,null_limit,active
0,ses_seguros.csv,ses_seguros,coenti,number,integer,no,yes,yes,ses_cias,NaN,NaN,NaN,NaN,yes
1,ses_cias.csv,ses_cias,coenti,number,integer,no,yes,no,NaN,NaN,NaN,NaN,NaN,yes
2,ses_ramos.csv,ses_ramos,coramo,number,integer,no,yes,no,NaN,NaN,NaN,NaN,NaN,yes


In [2]:
# Loop nos campos
df_data = pd.DataFrame([{}])
FAIL_TABLES = set()   # Controla falhas estruturais (Passo 1)
loaded_table = ""
separator = df_config.loc[0, "separator"]
encode = df_config.loc[0, "encode"]

for index, row in df_fields.iterrows():

    table_name = row["file"]
    file_path = df_config.loc[0, "data_path"] + table_name


    # Verifica se o arquivo está cqarregado ou já falhou
    if (row["file"] ==  loaded_table) or (table_name in FAIL_TABLES):
        continue
    else: 
        # Carrega o arquivo 
        load_data(df_data,file_path,separator,encode)
        loaded_table = table_name

    # Corrige o tipo das colunas inteiras exibidas como float 
    for col in df_data.select_dtypes(include=["float"]).columns:
        if (df_data[col].dropna() % 1 == 0).all():
            df_data[col] = df_data[col].astype("Int64") 

    print(loaded_table)


ses_seguros.csv
ses_cias.csv
ses_ramos.csv
